# Trabajo Práctico 2 - Matrices Insumo-Producto


## Imports.

In [23]:
import numpy as np
import pandas as pd
from numpy import linalg as LA
import scipy.linalg as sc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from IPython.display import display, HTML

import funciones as f
import funcionesTP1 as ftp1

AttributeError: 'numpy.ndarray' object has no attribute 'values'

## Consigna 2 - Graficamos los vectores $a_1$ y $a_2$, definidos con un tamaño de 250.

Creamos las matrices $A_1$ y $A_2$ tal como se describen en la consiga.

In [25]:
print("Matriz A1:\n")
print(f.A1)
print("Matriz A2:\n")
print(f.A2)

Matriz A1:



NameError: name 'f' is not defined

### Gráfica de la Norma del Vector por Posición

Realizamos el gráfico de puntos para cada posición i del vector, utilizando la función `graficarVector(matriz, tamaño), que devuelve un gráfico correspondiente.

La función `graficarVector` invoca a `vectorDeA_n`. Esta última calcula un vector donde cada posición i del vector representa la norma 2 de la matriz elevada a la potencia i:

posición i del vector: $\| A^i \|_2$

La función `vectorDeA_n(n, matrizOriginal)` devuelve un vector de dimensión n.


In [ ]:
f.graficarVector(f.A1, 250)

In [ ]:
f.graficarVector(f.A2, 250)

A partir de lo demostrado en el punto 1, si una matriz tiende a cero en un límite específico, su norma 2 también tenderá a cero en ese mismo límite, dado que sus elementos se vuelven cada vez más pequeños. Esto implica que las combinaciones lineales que representan los valores singulares de la matriz también disminuirán en magnitud. Sin embargo, si la matriz contiene algunos elementos no nulos, aunque sean pequeños, estos podrían influir en el valor singular más grande, lo que podría alterar la convergencia esperada.

Considerando estas condiciones y excepciones, podemos inferir que, en principio, la matriz $A_2$ parece converger, ya que su norma 2 se aproxima al cero. En contraste, la matriz $A_1$ no muestra signos de convergencia hacia cero después de 250 iteraciones, lo que sugiere que este número de iteraciones podría no ser suficiente para determinar su convergencia. Es importante tener en cuenta que la convergencia no solo depende de si los elementos de la matriz se acercan a cero, sino también de la composición global y la distribución de los valores dentro de la matriz.

## Consigna 3

### Implementación del método de la potencia para las matrices $A_1$ y $A_2$

Esta función recibe un vector inicial $v_0$ y multiplica la matriz A por $v_0$ en la primera iteración. El resultado se almacena y, de manera recursiva, se calculan los siguientes estados durante k iteraciones, normalizando el vector en cada paso para evitar un crecimiento excesivo y mantener la dirección del cambio. De este modo, se obtiene el autovector asociado.

La función `metodoPotencia(A, v, k)` devuelve el mayor autovalor y su autovector correspondiente. El cálculo se realiza mediante la siguiente ecuación:

$autovalor = \frac{(v^T A v)}{(v^T v)}$

Adicionalmente, utilizando el método de Monte Carlo, generamos 250 vectores iniciales $v_0$ aleatorios con valores entre 0 y 1000. Para cada uno de estos vectores, aplicamos el método de la potencia con k = 8. También calculamos el promedio de los autovalores obtenidos con cada $v_0$ y su desviación estándar.

La función `monteCarlo(A)` devuelve el promedio del mayor autovalor calculado en cada iteración, junto con su desviación estándar.


In [ ]:
prom_a1 ,DE_a1 = f.monteCarlo(f.A1) 
prom_a2 ,DE_a2 = f.monteCarlo(f.A2) 
tabla = pd.DataFrame({"Promedio":[prom_a1,prom_a2],"Desvio Estandar":[DE_a1,DE_a2]})
tabla.index = ["A1","A2"]
tabla

Se ejecutó el método de la potencia un total de 8 veces, observándose que los valores presentan un pequeño desvío respecto al promedio, de aproximadamente $10^{-5}$ para la matriz $A_1$ y $10^{-4}$ para la matriz $A_2$. Con base en estos resultados, se concluye que el autovalor encontrado para la matriz $A_1$ es 1, mientras que para la matriz $A_2$ es 0.927.


## Consigna 4 

In [ ]:
matrizConsigna4 = np.linalg.inv(np.identity(f.A1.shape[0]) - f.A1)

f.graficarVector(matrizConsigna4, 10)
f.vectorDeA_n(10, matrizConsigna4)

##ESTA MAAAL

## Consigna 5

- Creo $ A^{pp} y A^{nn} $ como en el TP1

In [ ]:
matriz = pd.read_excel("matriz.xlsx", sheet_name ="LAC_IOT_2011",)
Nic_col = []    
Pry_col = []
for i in range(1,41): #Crea la lista de columnas a filtrar
    Nic_col.append('NICs'+str(i))
    Pry_col.append('PRYs'+str(i))
    
Pry = matriz[matriz["Country_iso3"] == "PRY"] # Crea la tabla con filas de PRY
Nic = matriz[matriz["Country_iso3"] == "NIC"] # Crea la tabla con filas de NIC
# Crea matrices intra-regionales
Pry_int= Pry.loc[:,Pry_col] 
Nic_int = Nic.loc[:,Nic_col]

- uso la funcion de metodo de la potencia con un vector aleatorio y lo comparo con los autovalores dados por __numpy.linalg.eig__

In [ ]:
vect_Pry = np.random.randint(0,1000,size = Pry_int.shape[0]) #Creo vector aleatorio
aval_Pry,avect_Pry = f.metodoPotencia(Pry_int.to_numpy(),vect_Pry,250) #Uso metodo de la potencia
eigval_Pry,eigvect_Pry = LA.eig(Pry_int.to_numpy())

vect_Nic = np.random.randint(0,1000,size = Nic_int.shape[0])
aval_Nic,avect_Nic = f.metodoPotencia(Nic_int.to_numpy(),vect_Nic,250)
eigval_Nic,eigvect_Nic = LA.eig(Nic_int.to_numpy())

tabla = pd.DataFrame({"Aproximado":[aval_Pry,aval_Nic],"Real":[eigval_Pry[0],eigval_Nic[0]]})
tabla.index = ["Pry","Nic"]
tabla

## Consigna 6

## Consigna 7

<center></h1>Creo C dada por:</center></h1>

<center></h1>$C = {\overline{A^{pp}}}^{t} \, \overline{A^{pp}}/(40-1)$</center></h1>

<center></h1>donde  $\; \overline{A^{pp}} = E_{40}A^{pp}$, y $E_{n}$ es dada por: $E_{n} = I_{n} - \frac{1}{n}ee^{t}$</center></h1>

<center></h1>y $e \in \mathbb{R}^{n}$ es un vector columna de todos 1 </center></h1>


In [ ]:
n = Pry_int.shape[0]
Id = np.identity(n)
e = np.ones(n)
En = Id - 1/n * (np.atleast_2d(e).T @ np.atleast_2d(e))
C = ((En @ Pry_int.to_numpy()).T @ (En @ Pry_int.to_numpy()))/(40-1) 

Defino la funcion para crear un vector aleatorio de norma 1 y una funcion usando el algoritmo de Hotelling, donde se itera de la siguiente forma:
$$
x_{1} = Cx_{0} / \| Cx_{0} \|
$$
$$
x_{k+1} = Cx_{k} / \| Cx_{k} \|
$$
Para encontrar el primer autovector con maximo autovalor.  
- Para obtener el autovalor se usa el coeficiente de Rayleigh $ \lambda = \frac{v_{1}^{t}Cv_{1}}{v_{1}^{t}v_{1}} $  
- Si tiene autovalor positivo, el criterio de parada va a ser cuando $\| x_{k+1}^{t}\ - x_{k} \|_{2} <$ _valor_ , donde _valor_ es 1e-6.  
- En cambio si su autovalor es negativo, se usa como criterio cuando $ |\lambda_{k}| - |\lambda_{k+1}|$ < _valor_ , esto se debe a que cuando el autovalor es negativo, el autovector va a oscilar entre un vector y su opuesto, por lo que la norma de la diferencia siempre va a ser 2. En cambio este segundo criterio si funciona ya que el autovalor si converge.


In [ ]:
def create(n): 
     x = np.random.normal(size=n)
     x -= x.mean()
     return x / np.linalg.norm(x)
    
def Hotelling(A,v,valor):
    while True:
        v_prev = v
        vEstrella_prev = f.vectorEstrella(v_prev)
        aval_prev = (vEstrella_prev @ A @ v_prev) / (vEstrella_prev @ v_prev)
        
        v = A @ v
        v = v / np.linalg.norm(v)

        vEstrella = f.vectorEstrella(v)
        aval = (vEstrella @ A @ v) / (vEstrella @ v)
        res = vEstrella - v_prev
        
        if (aval >=0) and (np.linalg.norm(res,2) < valor):
            return aval,v 
            
        elif (aval < 0) and ((abs(aval)-abs(aval_prev)) < valor):
            return aval,v

Saco $v_{1}$ y $\lambda_{1}$ con la funcion Hotelling y comparo con lo dado por __numpy.linalg.eig__ 

In [ ]:
valor = 1e-10
vect = create(n)
avalH, vectH = Hotelling(C,vect,valor)
eigval, eigvect = LA.eig(C)
tabla = pd.DataFrame({"Aproximado":[vectH,avalH],"Real":[eigvect[:,0],eigval[0]]})
tabla.index = ["Autovector","Autovalor"]
display(HTML(tabla.to_html()))

Para encontrar $v_{2}$ y $\lambda_{2}$, se necesita encontrar $C'$, dada por:
$$
C' = C - \lambda_{1}v_{1}v_{1}^{t}
$$
Se vuelve a calcular Hotelling con $C'$ y otro vector aleatorio de norma 1 y se comparan los resultados con los dados por __numpy.linalg.eig__ 

In [ ]:
vectHEstrella = f.vectorEstrella(vectH)
C2 = C - avalH * (vectH @ vectHEstrella)
vect2 = create(n)
avalH2, vectH2 = Hotelling(C2,vect2,valor)
eigval2,eigvect2 = LA.eig(C2)
tabla = pd.DataFrame({"Aproximado":[vectH2,avalH2],"Real":[eigvect2[:,0],eigval2[0]]})
tabla.index = ["Autovector","Autovalor"]
display(HTML(tabla.to_html()))

## Consigna 8

In [ ]:
def proyectar(A,v1,v2):
    V = np.column_stack((v1,v2))
    return A @ V

Arr_proyectada = proyectar(Pry_int.to_numpy(),vectH,vectH2)


kmeans = KMeans(n_clusters=3, random_state=42)  # Busco clusters con KMeans
clusters = kmeans.fit_predict(Arr_proyectada)

df_clusters = pd.DataFrame(Arr_proyectada, columns=['v1', 'v2'])
df_clusters['Cluster'] = clusters

plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_clusters, x='v1', y='v2', hue='Cluster', palette='viridis', s=100)
plt.grid(True)
plt.show()

## Consigna 10

El analisis lo vamos a llevar a cabo sobre la matriz creada en el TP1, creada con las matrices regionales.

In [ ]:
A = ftp1.crearMatrizA()
H = A @ np.linalg.inv(np.identity(A.shape[0]) - A)

Realizamos el calculo de ACP por el metodo de la potencia implementado en la consigna 7. Lo realizamos de manera iterativa para poder conseguir la matriz V, base de autovectores. Pero como solo utilizaremos los primeros dos autovectores, no calculamos la V completa

In [ ]:
def centrarDatos(matriz):
    # Centramos los datos
    d, n = matriz.shape
    m = np.mean(matriz.values, axis=1)  # Asegúrate de que m sea un array de numpy

    X = matriz.values - np.tile(m.reshape((len(m), 1)), (1, n))  # Usa data.values para obtener el array de numpy
    Mcov = np.dot(X, X.T) / n  # Covariance Matrix
    return X, Mcov

def PrimerasNColumnasDeV_con_mP(matriz, n):
    d, _ = matriz.shape
    V = []  # Lista para almacenar los autovectores
    matriz_iterada = matriz.copy()

    for i in range(n):
        # Crear un vector inicial aleatorio
        vect = create(d)
        
        # Usamos Hotelling para obtener el autovector
        aval, vect = Hotelling(matriz_iterada, vect, valor)
        
        # Normalizamos el autovector
        vectEstrella = f.vectorEstrella(vect)
        
        # Actualizamos la matriz iterada (descontamos el componente encontrado)
        matriz_iterada -= aval * np.outer(vect, vectEstrella)
        
        # Añadimos el autovector encontrado a la lista
        V.append(vect)
    
    return np.array(V).T  # Devolvemos las primeras N columnas de V como una matriz

X, Mcov = centrarDatos(H)
V1 = PrimerasNColumnasDeV_con_mP(Mcov,2)

Realizamos un grafico proyectando los datos en 2-D, proyectando los datos en las dos direcciones de mayor cambio. 
Ademas, agregar el metodo de agrupamiento K-means al grafico, con n_clusters = 3.

In [ ]:
# Proyectar los datos en las dos primeras componentes principales
X_reducido = X.T @ V[:, :2]  # Transponer X para que las observaciones sean filas

# Aplicar K-means para encontrar clusters
kmeans = KMeans(n_clusters=3, random_state=42)  # Puedes ajustar el número de clusters
clusters = kmeans.fit_predict(X_reducido)

# Convertir el resultado a un DataFrame para facilitar la visualización
df_clusters = pd.DataFrame(X_reducido, columns=['Componente 1', 'Componente 2'])
df_clusters['Cluster'] = clusters

# Graficar los resultados
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df_clusters, x='Componente 1', y='Componente 2', hue='Cluster', palette='viridis', s=100)
plt.title('Clusters en el espacio de componentes principales')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

Vemos que el analisis de K-means no es el ideal ya que tenemos outliers, y ademas los datos no se llega a identificar grupos ocultos en los datos o intuidos, sino que la densidad de distribucion de los mismos.

Incluso realizamos un analisis de DBSCAN que es robusto detectando outliers y puede encontrar clusters con formas geometricas arbitrarias, pero no obtenemos mejores resultados. 
El analisis de DBCAN identifico un solo cluster(el asignado con el numero cero) y el resto outliers (los valores violetas en -1)

In [ ]:
# Supongamos que ya tienes X_reducido de tu análisis PCA
# Ajustar y aplicar DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan = dbscan.fit_predict(X_reducido)

# Convertir el resultado a un DataFrame para facilitar la visualización
df_dbscan = pd.DataFrame(X_reducido, columns=['Componente 1', 'Componente 2'])
df_dbscan['Cluster'] = clusters_dbscan

# Graficar los resultados de DBSCAN
plt.figure(figsize=(6, 4))
palette = sns.color_palette("viridis", as_cmap=True)
sns.scatterplot(data=df_dbscan, x='Componente 1', y='Componente 2', hue='Cluster', palette=palette, s=100)
plt.title('Clusters en el espacio de componentes principales (DBSCAN)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()

# Análisis de ruido
n_noise = np.sum(clusters_dbscan == -1)
total_points = df_dbscan.shape[0]
print(f"Número de puntos ruido: {n_noise} de {total_points} ({(n_noise / total_points) * 100:.2f}%)")

In [ ]:
# Paso 1: Contar los puntos en cada cluster
cluster_counts = df_dbscan['Cluster'].value_counts()

# Paso 2: Identificar clusters robustos (aquellos con más de una observación)
clusters_to_keep = cluster_counts[cluster_counts > 1].index

# Paso 3: Filtrar el DataFrame para mantener solo los clusters robustos
df_filtered = df_dbscan[df_clusters['Cluster'].isin(clusters_to_keep)]

# Paso 4: Graficar los clusters sin los outliers
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df_filtered, x='Componente 1', y='Componente 2', hue='Cluster', palette='viridis', s=100)
plt.title('Clusters sin Outliers')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.grid(True)
plt.show()


### Consigna 11

In [ ]:
def distanciaOrigen(punto):
    return np.sqrt(punto[0]**2 + punto[1]**2)

